In [2]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 19.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=2dc319bad8ba62c99f61e7488d00a41a8c4ceae159067e2c865b4429035559a0
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [3]:
#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [4]:
#Create a Spark Session
SpSession = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

In [25]:
# Load the file into a RDD
ccRaw = SpContext.textFile('/content/drive/My Drive/credit-card-default-1000.csv')
ccRaw.take(5)

['CUSTID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULTED',
 '530,20000,2,2,2,21,-1,-1,2,2,-2,-2,0,0,0,0,0,0,0,0,0,0,162000,0,0',
 '38,60000,2,2,2,22,0,0,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,1576,0',
 '43,10000,1,2,2,22,0,0,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,1500,0',
 '47,20000,2,1,2,22,0,0,2,-1,0,-1,1131,291,582,291,0,291,291,582,0,0,130291,651,0']

In [26]:
# Remove header row
dataLines = ccRaw.filter(lambda x: 'CUSTID' not in x)
dataLines.count()

1002

In [27]:
dataLines.take(5)

['530,20000,2,2,2,21,-1,-1,2,2,-2,-2,0,0,0,0,0,0,0,0,0,0,162000,0,0',
 '38,60000,2,2,2,22,0,0,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,1576,0',
 '43,10000,1,2,2,22,0,0,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,1500,0',
 '47,20000,2,1,2,22,0,0,2,-1,0,-1,1131,291,582,291,0,291,291,582,0,0,130291,651,0',
 '70,20000,1,4,2,22,2,0,0,0,-1,-1,1692,13250,433,1831,0,2891,13250,433,1831,0,2891,153504,0']

In [28]:

# Cleanup data. Remove the last two lines which are not 'CSV'
cleanedLines = dataLines.filter(lambda x: x.find('aaa') == -1)
cleanedLines.count()

1000

In [30]:
from pyspark.sql import Row
import numpy as np

# Convert into SQL Dataframe. In the process perform a few cleanups and changes required for future work
def convertToRow(instr):
    attList = instr.split(',')
    
    # PR#06 Round of age to range of 10s
    ageRound = round(float(attList[5]) / 10) * 10
    
    # Normalize sex to only 1 and 2
    sex = float(attList[2].replace('M', '1').replace('F', '2'))
    
    # Find average billed amount
    avgBillAmt = np.array(attList[12:18], dtype = 'float').mean().item()
    
    # Find average pay amount
    avgPayAmt = np.array(attList[18:24], dtype = 'float').mean().item()
    
    # Find average pay duration, required for PR#04
    # Make sure numbers are rounded and negative values are eliminated
    payDuration = np.array([x if float(x) > 0 else 0 for x in attList[6:12]], dtype = 'float')
    avgPayDuration = round(payDuration.mean().item())
    
    # Average percentage paid. Add this as an additional field to see if this field has any predictive capabilities
    perPay = round((avgPayAmt / (avgBillAmt + 1) * 100) / 25) * 25
    
    values = Row(CUSTID = attList[0], LIMIT_BAL = float(attList[1]), SEX = sex, EDUCATION = float(attList[3]), \
                 MARRIAGE = float(attList[4]), AGE = ageRound, AVG_PAY_DUR = avgPayDuration, \
                 AVG_BILL_AMT = avgBillAmt, AVG_PAY_AMT = avgPayAmt, PER_PAID = perPay, DEFAULTED = float(attList[24]))
    
    return values

In [31]:
# Cleanedup RDD
ccRows = cleanedLines.map(convertToRow)
ccRows.take(5)

[Row(CUSTID='530', LIMIT_BAL=20000.0, SEX=2.0, EDUCATION=2.0, MARRIAGE=2.0, AGE=20, AVG_PAY_DUR=1, AVG_BILL_AMT=0.0, AVG_PAY_AMT=27000.0, PER_PAID=2700000, DEFAULTED=0.0),
 Row(CUSTID='38', LIMIT_BAL=60000.0, SEX=2.0, EDUCATION=2.0, MARRIAGE=2.0, AGE=20, AVG_PAY_DUR=0, AVG_BILL_AMT=0.0, AVG_PAY_AMT=262.6666666666667, PER_PAID=26275, DEFAULTED=0.0),
 Row(CUSTID='43', LIMIT_BAL=10000.0, SEX=1.0, EDUCATION=2.0, MARRIAGE=2.0, AGE=20, AVG_PAY_DUR=0, AVG_BILL_AMT=0.0, AVG_PAY_AMT=250.0, PER_PAID=25000, DEFAULTED=0.0),
 Row(CUSTID='47', LIMIT_BAL=20000.0, SEX=2.0, EDUCATION=1.0, MARRIAGE=2.0, AGE=20, AVG_PAY_DUR=0, AVG_BILL_AMT=431.0, AVG_PAY_AMT=21969.166666666668, PER_PAID=5075, DEFAULTED=0.0),
 Row(CUSTID='70', LIMIT_BAL=20000.0, SEX=1.0, EDUCATION=4.0, MARRIAGE=2.0, AGE=20, AVG_PAY_DUR=0, AVG_BILL_AMT=3349.5, AVG_PAY_AMT=28651.5, PER_PAID=850, DEFAULTED=0.0)]

In [32]:
# Create a dataframe
ccDf = SpSession.createDataFrame(ccRows)
ccDf.cache()
ccDf.show(10)

+------+---------+---+---------+--------+---+-----------+------------------+------------------+--------+---------+
|CUSTID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|AVG_PAY_DUR|      AVG_BILL_AMT|       AVG_PAY_AMT|PER_PAID|DEFAULTED|
+------+---------+---+---------+--------+---+-----------+------------------+------------------+--------+---------+
|   530|  20000.0|2.0|      2.0|     2.0| 20|          1|               0.0|           27000.0| 2700000|      0.0|
|    38|  60000.0|2.0|      2.0|     2.0| 20|          0|               0.0| 262.6666666666667|   26275|      0.0|
|    43|  10000.0|1.0|      2.0|     2.0| 20|          0|               0.0|             250.0|   25000|      0.0|
|    47|  20000.0|2.0|      1.0|     2.0| 20|          0|             431.0|21969.166666666668|    5075|      0.0|
|    70|  20000.0|1.0|      4.0|     2.0| 20|          0|            3349.5|           28651.5|     850|      0.0|
|    79|  30000.0|2.0|      2.0|     2.0| 20|          0|1025.3333333333333|    

In [33]:
ccDf.printSchema()


root
 |-- CUSTID: string (nullable = true)
 |-- LIMIT_BAL: double (nullable = true)
 |-- SEX: double (nullable = true)
 |-- EDUCATION: double (nullable = true)
 |-- MARRIAGE: double (nullable = true)
 |-- AGE: long (nullable = true)
 |-- AVG_PAY_DUR: long (nullable = true)
 |-- AVG_BILL_AMT: double (nullable = true)
 |-- AVG_PAY_AMT: double (nullable = true)
 |-- PER_PAID: long (nullable = true)
 |-- DEFAULTED: double (nullable = true)



In [34]:
import pandas as pd
# Add SEX_NAME to the data using SQL join. Required for PR#02
genderDf = SpSession.createDataFrame(pd.DataFrame({'SEX': [1.0, 2.0], 'SEX_NAME': ['Male', 'Female']}))
genderDf.show()

+---+--------+
|SEX|SEX_NAME|
+---+--------+
|1.0|    Male|
|2.0|  Female|
+---+--------+



In [35]:
ccDf1 = ccDf.join(genderDf, ccDf.SEX == genderDf.SEX).drop(genderDf.SEX)
ccDf1.show(5)

+------+---------+---+---------+--------+---+-----------+-------------------+-----------------+--------+---------+--------+
|CUSTID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|AVG_PAY_DUR|       AVG_BILL_AMT|      AVG_PAY_AMT|PER_PAID|DEFAULTED|SEX_NAME|
+------+---------+---+---------+--------+---+-----------+-------------------+-----------------+--------+---------+--------+
|   388|  80000.0|1.0|      3.0|     1.0| 70|          0| -65.66666666666667|416.6666666666667|    -650|      1.0|    Male|
|   103| 480000.0|1.0|      1.0|     1.0| 60|          0|-56043.166666666664|          57956.5|    -100|      1.0|    Male|
|   932| 320000.0|1.0|      1.0|     1.0| 60|          0|                0.0|              0.0|       0|      1.0|    Male|
|   948|  50000.0|1.0|      2.0|     1.0| 60|          0|                0.0|              0.0|       0|      1.0|    Male|
|   602|  30000.0|1.0|      3.0|     1.0| 60|          0| 25828.333333333332|              0.0|       0|      1.0|    Male|
+------+

In [36]:
# Add ED_STR to the data using SQL join. Required for PR#03
eduDf = SpSession.createDataFrame(pd.DataFrame({'EDUCATION': [1.0, 2.0, 3.0, 4.0], 
                                                'ED_STR': ['Graduate', 'University', 'High School', 'Others']}))
eduDf.show()

+---------+-----------+
|EDUCATION|     ED_STR|
+---------+-----------+
|      1.0|   Graduate|
|      2.0| University|
|      3.0|High School|
|      4.0|     Others|
+---------+-----------+



In [37]:
ccDf2 = ccDf1.join(eduDf, ccDf1.EDUCATION == eduDf.EDUCATION).drop(eduDf.EDUCATION)
ccDf2.show(5)

+------+---------+---+---------+--------+---+-----------+-------------------+-----------+--------+---------+--------+--------+
|CUSTID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|AVG_PAY_DUR|       AVG_BILL_AMT|AVG_PAY_AMT|PER_PAID|DEFAULTED|SEX_NAME|  ED_STR|
+------+---------+---+---------+--------+---+-----------+-------------------+-----------+--------+---------+--------+--------+
|   103| 480000.0|1.0|      1.0|     1.0| 60|          0|-56043.166666666664|    57956.5|    -100|      1.0|    Male|Graduate|
|   932| 320000.0|1.0|      1.0|     1.0| 60|          0|                0.0|        0.0|       0|      1.0|    Male|Graduate|
|   466| 230000.0|1.0|      1.0|     1.0| 60|          0|                0.0|        0.0|       0|      1.0|    Male|Graduate|
|    35| 500000.0|1.0|      1.0|     1.0| 60|          1|            19763.0|        0.0|       0|      1.0|    Male|Graduate|
|    66| 200000.0|1.0|      1.0|     1.0| 60|          0|              -87.5|        0.0|       0|      1.0|   

In [38]:
# Add MARR_DESC to the data with SQL join. Required for PR#03
marrDf = SpSession.createDataFrame(pd.DataFrame({'MARRIAGE': [1.0, 2.0, 3.0], 
                                                 'MARR_DESC': ['Single', 'Married', 'Others']}))
marrDf.show()

+--------+---------+
|MARRIAGE|MARR_DESC|
+--------+---------+
|     1.0|   Single|
|     2.0|  Married|
|     3.0|   Others|
+--------+---------+



In [39]:
ccFinalDf = ccDf2.join(marrDf, ccDf2.MARRIAGE == marrDf.MARRIAGE).drop(marrDf.MARRIAGE)
ccFinalDf.show(5)

+------+---------+---+---------+--------+---+-----------+-------------------+-----------+--------+---------+--------+--------+---------+
|CUSTID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|AVG_PAY_DUR|       AVG_BILL_AMT|AVG_PAY_AMT|PER_PAID|DEFAULTED|SEX_NAME|  ED_STR|MARR_DESC|
+------+---------+---+---------+--------+---+-----------+-------------------+-----------+--------+---------+--------+--------+---------+
|   103| 480000.0|1.0|      1.0|     1.0| 60|          0|-56043.166666666664|    57956.5|    -100|      1.0|    Male|Graduate|   Single|
|   932| 320000.0|1.0|      1.0|     1.0| 60|          0|                0.0|        0.0|       0|      1.0|    Male|Graduate|   Single|
|   466| 230000.0|1.0|      1.0|     1.0| 60|          0|                0.0|        0.0|       0|      1.0|    Male|Graduate|   Single|
|    35| 500000.0|1.0|      1.0|     1.0| 60|          1|            19763.0|        0.0|       0|      1.0|    Male|Graduate|   Single|
|    66| 200000.0|1.0|      1.0|     1.0|

In [40]:
# create a temp view
ccFinalDf.createOrReplaceTempView('CCDATA')

In [41]:
# PR#02 solution
SpSession.sql('SELECT SEX_NAME, count(*) AS Total, SUM(DEFAULTED) AS Defaults, ' + \
             'ROUND(SUM(DEFAULTED) * 100 / count(*)) AS PER_DEFAULT FROM CCDATA GROUP BY SEX_NAME').show()

+--------+-----+--------+-----------+
|SEX_NAME|Total|Defaults|PER_DEFAULT|
+--------+-----+--------+-----------+
|  Female|  591|   218.0|       37.0|
|    Male|  409|   185.0|       45.0|
+--------+-----+--------+-----------+



In [42]:
# PR#03 solution
SpSession.sql('SELECT MARR_DESC, ED_STR, COUNT(*) AS Total, SUM(DEFAULTED) AS Defaults, ' + \
             'ROUND(SUM(DEFAULTED) * 100 / count(*)) AS PER_DEFAULT FROM CCDATA GROUP BY MARR_DESC, ED_STR ' + \
             'ORDER BY 1, 2').show()

+---------+-----------+-----+--------+-----------+
|MARR_DESC|     ED_STR|Total|Defaults|PER_DEFAULT|
+---------+-----------+-----+--------+-----------+
|  Married|   Graduate|  268|    69.0|       26.0|
|  Married|High School|   55|    24.0|       44.0|
|  Married|     Others|    4|     2.0|       50.0|
|  Married| University|  243|    65.0|       27.0|
|   Others|   Graduate|    4|     4.0|      100.0|
|   Others|High School|    8|     6.0|       75.0|
|   Others| University|    7|     3.0|       43.0|
|   Single|   Graduate|  123|    71.0|       58.0|
|   Single|High School|   87|    52.0|       60.0|
|   Single|     Others|    3|     2.0|       67.0|
|   Single| University|  198|   105.0|       53.0|
+---------+-----------+-----+--------+-----------+



In [43]:
# PR#04 solution
SpSession.sql('SELECT AVG_PAY_DUR, COUNT(*) AS Total, SUM(DEFAULTED) AS Defaults, ' + \
             'ROUND(SUM(DEFAULTED) * 100 / count(*)) AS PER_DEFAULT FROM CCDATA GROUP BY AVG_PAY_DUR ORDER BY 1').show()

+-----------+-----+--------+-----------+
|AVG_PAY_DUR|Total|Defaults|PER_DEFAULT|
+-----------+-----+--------+-----------+
|          0|  786|   290.0|       37.0|
|          1|  192|   102.0|       53.0|
|          2|   19|    10.0|       53.0|
|          3|    3|     1.0|       33.0|
+-----------+-----+--------+-----------+



In [45]:
from pyspark.ml.linalg import Vectors


In [46]:
def transformToLabeledPoint(row):
    lp = (row['DEFAULTED'], Vectors.dense([row['AGE'], row['AVG_BILL_AMT'], row['AVG_PAY_AMT'], row['AVG_PAY_DUR'], \
                                          row['EDUCATION'], row['LIMIT_BAL'], row['MARRIAGE'], row['PER_PAID'], \
                                          row['SEX']]))
    return lp

In [47]:
ccLp = ccFinalDf.rdd.map(transformToLabeledPoint)
ccLp.take(5)

[(1.0,
  DenseVector([60.0, -56043.1667, 57956.5, 0.0, 1.0, 480000.0, 1.0, -100.0, 1.0])),
 (1.0, DenseVector([60.0, 0.0, 0.0, 0.0, 1.0, 320000.0, 1.0, 0.0, 1.0])),
 (1.0, DenseVector([60.0, 0.0, 0.0, 0.0, 1.0, 230000.0, 1.0, 0.0, 1.0])),
 (1.0, DenseVector([60.0, 19763.0, 0.0, 1.0, 1.0, 500000.0, 1.0, 0.0, 1.0])),
 (1.0, DenseVector([60.0, -87.5, 0.0, 0.0, 1.0, 200000.0, 1.0, 0.0, 1.0]))]

In [48]:
ccNormDf = SpSession.createDataFrame(ccLp, ['label', 'features'])
ccNormDf.select('*').show(10, truncate = False)
ccNormDf.cache()

+-----+------------------------------------------------------------------+
|label|features                                                          |
+-----+------------------------------------------------------------------+
|1.0  |[60.0,-56043.166666666664,57956.5,0.0,1.0,480000.0,1.0,-100.0,1.0]|
|1.0  |[60.0,0.0,0.0,0.0,1.0,320000.0,1.0,0.0,1.0]                       |
|1.0  |[60.0,0.0,0.0,0.0,1.0,230000.0,1.0,0.0,1.0]                       |
|1.0  |[60.0,19763.0,0.0,1.0,1.0,500000.0,1.0,0.0,1.0]                   |
|1.0  |[60.0,-87.5,0.0,0.0,1.0,200000.0,1.0,0.0,1.0]                     |
|1.0  |[60.0,125.0,0.0,1.0,1.0,50000.0,1.0,0.0,1.0]                      |
|1.0  |[50.0,0.0,0.0,0.0,1.0,170000.0,1.0,0.0,1.0]                       |
|1.0  |[50.0,1270.6666666666667,0.0,0.0,1.0,290000.0,1.0,0.0,1.0]        |
|1.0  |[50.0,0.0,0.0,1.0,1.0,600000.0,1.0,0.0,1.0]                       |
|1.0  |[50.0,81.83333333333333,0.0,1.0,1.0,200000.0,1.0,0.0,1.0]         |
+-----+------------------

DataFrame[label: double, features: vector]

In [49]:
# Indexing needed as pre-req
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol = 'label', outputCol = 'indexed')
si_model = stringIndexer.fit(ccNormDf)
td = si_model.transform(ccNormDf)
td.show(5)

+-----+--------------------+-------+
|label|            features|indexed|
+-----+--------------------+-------+
|  1.0|[60.0,-56043.1666...|    1.0|
|  1.0|[60.0,0.0,0.0,0.0...|    1.0|
|  1.0|[60.0,0.0,0.0,0.0...|    1.0|
|  1.0|[60.0,19763.0,0.0...|    1.0|
|  1.0|[60.0,-87.5,0.0,0...|    1.0|
+-----+--------------------+-------+
only showing top 5 rows



In [50]:
# split into training and testing data
trainingData, testData = td.randomSplit([0.7, 0.3])

In [51]:
trainingData.count()


704

In [52]:
testData.count()


296

In [54]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol = 'prediction', labelCol = 'indexed', 
                                              metricName = 'accuracy')

# Create the Dicision Tree model (with hyper-parameter tuning)
paramGrid = ParamGridBuilder().addGrid(DecisionTreeClassifier.maxDepth, [4, 5, 6]).build()

dtClassifier = DecisionTreeClassifier(labelCol = 'indexed', featuresCol = 'features')

dtClassifier_cv = CrossValidator(estimator = DecisionTreeClassifier(), estimatorParamMaps = paramGrid,
                                evaluator = MulticlassClassificationEvaluator(), numFolds = 2)


dtModel = dtClassifier_cv.fit(trainingData)

# Predict on the test data
predictions = dtModel.transform(testData)
predictions.select('prediction', 'indexed', 'label', 'features').show()

+----------+-------+-----+--------------------+
|prediction|indexed|label|            features|
+----------+-------+-----+--------------------+
|       1.0|    0.0|  0.0|[20.0,12.83333333...|
|       0.0|    0.0|  0.0|[20.0,117.8333333...|
|       0.0|    0.0|  0.0|[20.0,415.0,415.0...|
|       0.0|    0.0|  0.0|[20.0,416.6666666...|
|       0.0|    0.0|  0.0|[20.0,6924.0,6751...|
|       0.0|    0.0|  0.0|[30.0,1596.666666...|
|       0.0|    0.0|  0.0|[30.0,3709.333333...|
|       1.0|    0.0|  0.0|[30.0,5737.166666...|
|       1.0|    0.0|  0.0|[30.0,8030.666666...|
|       0.0|    0.0|  0.0|[30.0,8769.0,6085...|
|       1.0|    0.0|  0.0|[30.0,9349.5,1049...|
|       1.0|    0.0|  0.0|[30.0,10652.0,855...|
|       0.0|    0.0|  0.0|[30.0,12778.16666...|
|       0.0|    0.0|  0.0|[30.0,14673.5,318...|
|       0.0|    0.0|  0.0|[30.0,14870.0,941...|
|       0.0|    0.0|  0.0|[30.0,15523.16666...|
|       0.0|    0.0|  0.0|[30.0,16942.16666...|
|       0.0|    0.0|  0.0|[30.0,17267.16

In [55]:
print('Results of Decision Tree: {}'.format(evaluator.evaluate(predictions)))

Results of Decision Tree: 0.722972972972973


In [56]:
#Create the Random Forest model
rfClassifier = RandomForestClassifier(labelCol = 'indexed', featuresCol = 'features')
rfModel = rfClassifier.fit(trainingData)

In [57]:
# Predict on the test data
predictions = rfModel.transform(testData)
predictions.select('prediction', 'indexed', 'label', 'features').show()

+----------+-------+-----+--------------------+
|prediction|indexed|label|            features|
+----------+-------+-----+--------------------+
|       0.0|    0.0|  0.0|[20.0,12.83333333...|
|       0.0|    0.0|  0.0|[20.0,117.8333333...|
|       0.0|    0.0|  0.0|[20.0,415.0,415.0...|
|       0.0|    0.0|  0.0|[20.0,416.6666666...|
|       0.0|    0.0|  0.0|[20.0,6924.0,6751...|
|       0.0|    0.0|  0.0|[30.0,1596.666666...|
|       0.0|    0.0|  0.0|[30.0,3709.333333...|
|       0.0|    0.0|  0.0|[30.0,5737.166666...|
|       0.0|    0.0|  0.0|[30.0,8030.666666...|
|       0.0|    0.0|  0.0|[30.0,8769.0,6085...|
|       0.0|    0.0|  0.0|[30.0,9349.5,1049...|
|       0.0|    0.0|  0.0|[30.0,10652.0,855...|
|       0.0|    0.0|  0.0|[30.0,12778.16666...|
|       0.0|    0.0|  0.0|[30.0,14673.5,318...|
|       0.0|    0.0|  0.0|[30.0,14870.0,941...|
|       0.0|    0.0|  0.0|[30.0,15523.16666...|
|       0.0|    0.0|  0.0|[30.0,16942.16666...|
|       0.0|    0.0|  0.0|[30.0,17267.16

In [58]:
print('Results of Random Forest: {}'.format(evaluator.evaluate(predictions)))


Results of Random Forest: 0.7398648648648649


In [59]:
# Filter only columns needed for clustering
ccClustDf = ccFinalDf.select('SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'CUSTID')

# Do centering and scaling for the values
summStats = ccClustDf.describe().toPandas()
summStats

,summary,SEX,EDUCATION,MARRIAGE,AGE,CUSTID
0,count,1000,1000,1000,1000,1000
1,mean,1.591,1.769,1.608,34.9,500.5
2,stddev,0.49189527438364455,0.7212206036707212,0.5259397423779768,10.084229851192974,288.8194360957493
3,min,1.0,1.0,1.0,20,1
4,max,2.0,4.0,3.0,80,999


In [60]:
meanValues = summStats.iloc[1, 1:].values.tolist()
stdValues = summStats.iloc[2, 1:].values.tolist()

bcMeans = SpContext.broadcast(meanValues)
bcStdDev = SpContext.broadcast(stdValues)

In [61]:
def centerAndScale(inRow):
    global bcMeans
    global bcStdDev
    
    meanArray = bcMeans.value
    stdArray = bcStdDev.value
    
    retArray = []
    for i in range(len(meanArray)):
        retArray.append((float(inRow[i]) - float(meanArray[i])) / float(stdArray[i]))
    return Row(CUSTID = inRow[4], features = Vectors.dense(retArray))

In [62]:
ccMap = ccClustDf.rdd.map(centerAndScale)
ccMap.take(5)

[Row(CUSTID='103', features=DenseVector([-1.2015, -1.0662, -1.156, 2.489, -1.3763])),
 Row(CUSTID='932', features=DenseVector([-1.2015, -1.0662, -1.156, 2.489, 1.494])),
 Row(CUSTID='466', features=DenseVector([-1.2015, -1.0662, -1.156, 2.489, -0.1195])),
 Row(CUSTID='35', features=DenseVector([-1.2015, -1.0662, -1.156, 2.489, -1.6117])),
 Row(CUSTID='66', features=DenseVector([-1.2015, -1.0662, -1.156, 2.489, -1.5044]))]

In [63]:
# Create a data frame with the features
ccFinalClustDf = SpSession.createDataFrame(ccMap)
ccFinalClustDf.cache()
ccFinalClustDf.show(10, truncate = False)

+------+---------------------------------------------------------------------------------------------------+
|CUSTID|features                                                                                           |
+------+---------------------------------------------------------------------------------------------------+
|103   |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,-1.376292417758966]  |
|932   |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,1.494013027076714]   |
|466   |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,-0.11945179474889138]|
|35    |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,-1.611733636394462]  |
|66    |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,-1.504400139663574]  |
|553   |[-1.2014752545458702,-1.0662479636412228,-1.156025968395157,2.489034896108665,0.1817744702700521]  |
|603   |[-1.2014752

In [64]:
# Perform clustering
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k = 4, seed = 1)
model = kmeans.fit(ccFinalClustDf)

In [65]:
predictions = model.transform(ccFinalClustDf)
predictions.select('*').show()

+------+--------------------+----------+
|CUSTID|            features|prediction|
+------+--------------------+----------+
|   103|[-1.2014752545458...|         1|
|   932|[-1.2014752545458...|         2|
|   466|[-1.2014752545458...|         1|
|    35|[-1.2014752545458...|         1|
|    66|[-1.2014752545458...|         1|
|   553|[-1.2014752545458...|         1|
|   603|[-1.2014752545458...|         2|
|   576|[-1.2014752545458...|         2|
|   452|[-1.2014752545458...|         2|
|    91|[-1.2014752545458...|         1|
|   406|[-1.2014752545458...|         2|
|   451|[-1.2014752545458...|         2|
|   395|[-1.2014752545458...|         2|
|    18|[-1.2014752545458...|         1|
|   917|[-1.2014752545458...|         2|
|   260|[-1.2014752545458...|         1|
|   724|[-1.2014752545458...|         2|
|   686|[-1.2014752545458...|         2|
|   207|[-1.2014752545458...|         1|
|   627|[-1.2014752545458...|         2|
+------+--------------------+----------+
only showing top